In [1]:
pip install shared_utils

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import branca
import folium
from shared_utils import gtfs_utils_v2

from siuba import *
import pandas as pd
import geopandas as gpd 

import datetime as dt
import time

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import sys
sys.path.append('../rt_segment_speeds/segment_speed_utils')

from helpers import *

In [4]:
stops_data = import_scheduled_stops("2022-06-01").drop_duplicates().reset_index(drop=True)

In [5]:
metro_stops = stops_data[stops_data['feed_key'] == '06d1f3ac2b0ae5e74424edbbfefa19ed']

In [6]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

In [7]:
mar_metro_raw = pd.read_excel(f'{GCS_FILE_PATH}rider_182_2022_03.xlsx')

In [8]:
mar_metro_raw = mar_metro_raw.iloc[:-2,:]

In [9]:
day_cols = {'DX': 'weekday_ons', 'SA': 'sat_ons', 'SU': 'sun_ons'}

mar_metro_grouped = (mar_metro_raw
                 >> mutate(STOP_ID = _.STOP_ID.astype('int64').astype(str))
                 >> mutate(DAY_TYPE = _.DAY_TYPE.apply(lambda x: day_cols[x]))
                 >> group_by(_.STOP_ID, _.STOP_NAME, _.DAY_TYPE)
                 >> summarize(stop_total_ons = _.Total_Ons.sum())
                )

In [10]:
yr_metro_grouped = (mar_metro_grouped
                     >> mutate(stop_total_ons = _.stop_total_ons*11.91221463) #make yearly: comes from fy22/mar22 NTD ridership
                     >> spread("DAY_TYPE", "stop_total_ons")
                     >> rename(stop_id = _.STOP_ID)
                     >> mutate(feed_key = '06d1f3ac2b0ae5e74424edbbfefa19ed')
                     >> mutate(name = 'LA Metro Bus Schedule')
                    )

In [11]:
stops_to_join = metro_stops >> select(_.feed_key, _.stop_id, _.stop_name, _.geometry)

In [12]:
yr_metro_joined = stops_to_join >> inner_join(_, yr_metro_grouped, on = ['feed_key', 'stop_id'])

In [13]:
yr_metro_joined.head(5)

,feed_key,stop_id,stop_name,geometry,STOP_NAME,sat_ons,sun_ons,weekday_ons,name
0,06d1f3ac2b0ae5e74424edbbfefa19ed,12591,LA Zoo,POINT (158199.490 -428414.858),LA ZOO,857.679453,381.190868,2775.546009,LA Metro Bus Schedule
1,06d1f3ac2b0ae5e74424edbbfefa19ed,5377,1st / Hill,POINT (161833.578 -438634.619),1ST / HILL,14914.092717,11531.023762,159742.798188,LA Metro Bus Schedule
2,06d1f3ac2b0ae5e74424edbbfefa19ed,15612,1st / Hill,POINT (161849.863 -438611.462),1ST / HILL,4800.622496,3383.068955,43551.056687,LA Metro Bus Schedule
3,06d1f3ac2b0ae5e74424edbbfefa19ed,1217,6th / Wall,POINT (161822.154 -439849.592),6TH / WALL,3502.191101,3061.439160,24682.108713,LA Metro Bus Schedule
4,06d1f3ac2b0ae5e74424edbbfefa19ed,7376,7th / Alma,POINT (157649.168 -473829.124),7TH / ALMA,0.000000,35.736644,559.874088,LA Metro Bus Schedule


In [14]:
stops_remainder = (stops_to_join 
    >> anti_join(_, yr_metro_grouped, on = ['feed_key', 'stop_id'])
                    )

len(stops_remainder)

5

In [15]:
stops_remainder >> distinct(_.stop_name, _.stop_id)

,stop_name,stop_id
0,Dodger Stadium Express - Union Station West Po...,2155
1,Dodger Express Loading Zone #3 (Harbor Gateway),63500003
2,Dodger Express Loading Zone #2 (Union Station),63500004
3,Dodger Express Loading Zone #1 (Union Station),63500001
4,Cesar E Chavez / Broadway,63500002


In [16]:
yr_metro_remainder = (yr_metro_grouped 
    >> anti_join(_, stops_to_join, on = ['feed_key', 'stop_id'])
                    )

len(yr_metro_remainder)

8

In [17]:
yr_metro_remainder

,stop_id,STOP_NAME,sat_ons,sun_ons,weekday_ons,feed_key,name
6324,20400011,71ST / CRENSHAW,845.767239,774.293951,NaN,06d1f3ac2b0ae5e74424edbbfefa19ed,LA Metro Bus Schedule
6819,2506,DO NOT ANNOUNCE THIS STOP!,0.000000,0.000000,47.648859,06d1f3ac2b0ae5e74424edbbfefa19ed,LA Metro Bus Schedule
6820,2507,DO NOT ANNOUNCE THIS STOP!,0.000000,11.912215,11.912215,06d1f3ac2b0ae5e74424edbbfefa19ed,LA Metro Bus Schedule
7022,2741,DO NOT ANNOUNCE THIS STOP!,47.648859,35.736644,238.244293,06d1f3ac2b0ae5e74424edbbfefa19ed,LA Metro Bus Schedule
7275,3034,DO NOT ANNOUNCE THIS STOP!,0.000000,0.000000,23.824429,06d1f3ac2b0ae5e74424edbbfefa19ed,LA Metro Bus Schedule
7486,3258,DO NOT ANNOUNCE THIS STOP!,59.561073,0.000000,262.068722,06d1f3ac2b0ae5e74424edbbfefa19ed,LA Metro Bus Schedule
7674,3513,ROSCOE / TOPANGA CANYON,571.786302,238.244293,NaN,06d1f3ac2b0ae5e74424edbbfefa19ed,LA Metro Bus Schedule
7706,3559,DO NOT ANNOUNCE THIS STOP!,47.648859,0.000000,23.824429,06d1f3ac2b0ae5e74424edbbfefa19ed,LA Metro Bus Schedule


In [18]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/ahsc_grant'
yr_metro_joined.to_parquet(f"{GCS_FILE_PATH}/ridership_metro_08_26_2024.parquet")